# Method-II (Time Integration)

In [134]:
import math
import numpy as np
import matplotlib.pyplot as plt
plt.ioff() # interactive mode off
# from IPython.display import Latex
# from matplotlib.patches import Rectangle
from matplotlib import animation
import pandas as pd
# plt.rcParams['figure.figsize']=(16,9)
plt.rcParams.update({'font.size': 13})
plt.rcParams['animation.html'] = 'html5'
# plt.rcParams['text.usetex'] = True
# %config InlineBackend.close_figures=False # keep figures open in pyplot

In [135]:
def postproc(x1, x2, x3, T=10, N=200, P=[1,1,1], E=2000000, nu=0.2, limit=[-12,12,-12,12], animx=1):
    """
    X1,X2,X3 - material coordinates
    x1,x2,x3 - spacial coordinates
    t - current time(sec)
    T - total time(sec)
    N - no of time steps
    P - point at which the F,C,b,E,e,sigma,S are evaluated
    """
    ## Shape Functions and Their derivatives w.r.to X1,X2,X3 -- Linear Hexahedral Element
    # ---------------------------------------------------------- Shape Functions
    def N1(X1,X2,X3): return 0.125*(1-X1)*(1+X2)*(1+X3)
    def N2(X1,X2,X3): return 0.125*(1+X1)*(1+X2)*(1+X3)
    def N3(X1,X2,X3): return 0.125*(1-X1)*(1-X2)*(1+X3)
    def N4(X1,X2,X3): return 0.125*(1+X1)*(1-X2)*(1+X3)
    def N5(X1,X2,X3): return 0.125*(1-X1)*(1+X2)*(1-X3)
    def N6(X1,X2,X3): return 0.125*(1+X1)*(1+X2)*(1-X3)
    def N7(X1,X2,X3): return 0.125*(1-X1)*(1-X2)*(1-X3)
    def N8(X1,X2,X3): return 0.125*(1+X1)*(1-X2)*(1-X3)
    # ---------------------------- Derivatives of the shape functions w.r.to 'X1'
    def N1X1(X2,X3): return -0.125*(1+X2)*(1+X3)
    def N2X1(X2,X3): return  0.125*(1+X2)*(1+X3)
    def N3X1(X2,X3): return -0.125*(1-X2)*(1+X3)
    def N4X1(X2,X3): return  0.125*(1-X2)*(1+X3)
    def N5X1(X2,X3): return -0.125*(1+X2)*(1-X3)
    def N6X1(X2,X3): return  0.125*(1+X2)*(1-X3)
    def N7X1(X2,X3): return -0.125*(1-X2)*(1-X3)
    def N8X1(X2,X3): return  0.125*(1-X2)*(1-X3)
    # ---------------------------- Derivatives of the shape functions w.r.to 'X2'
    def N1X2(X1,X3): return  0.125*(1-X1)*(1+X3)
    def N2X2(X1,X3): return  0.125*(1+X1)*(1+X3)
    def N3X2(X1,X3): return -0.125*(1-X1)*(1+X3)
    def N4X2(X1,X3): return -0.125*(1+X1)*(1+X3)
    def N5X2(X1,X3): return  0.125*(1-X1)*(1-X3)
    def N6X2(X1,X3): return  0.125*(1+X1)*(1-X3)
    def N7X2(X1,X3): return -0.125*(1-X1)*(1-X3)
    def N8X2(X1,X3): return -0.125*(1+X1)*(1-X3)
    # ---------------------------- Derivatives of the shape functions w.r.to 'X3'
    def N1X3(X1,X2): return  0.125*(1-X1)*(1+X2)
    def N2X3(X1,X2): return  0.125*(1+X1)*(1+X2)
    def N3X3(X1,X2): return  0.125*(1-X1)*(1-X2)
    def N4X3(X1,X2): return  0.125*(1+X1)*(1-X2)
    def N5X3(X1,X2): return -0.125*(1-X1)*(1+X2)
    def N6X3(X1,X2): return -0.125*(1+X1)*(1+X2)
    def N7X3(X1,X2): return -0.125*(1-X1)*(1-X2)
    def N8X3(X1,X2): return -0.125*(1+X1)*(1-X2)

    def eqstress(S):
        Sd = S - (S.trace()/3)*I
        Seq = math.sqrt(1.5*np.tensordot(Sd,Sd))
        return Seq
    
    def eqstrain(E,nu):
        dummy = ((E[0,0]-E[1,1])**2 + E[1,1]-E[2,2])**2 + (E[2,2]-E[0,0])**2 + 3*(E[0,1]**2+E[1,2]**2+E[2,0]**2)
        Eeq = (1/(math.sqrt(2)*(1+nu)))*math.sqrt(dummy)
        return Eeq

    dt = T/N
    lmda = (E*nu)/((1+nu)*(1-2*nu))
    mu   = 1/(2*(1+nu))
    I = np.array([[1,0,0],[0,1,0],[0,0,1]])

    nodes_t0 = np.array([[-1,1,1],[1,1,1],[-1,-1,1],[1,-1,1],[-1,1,-1],[1,1,-1],[-1,-1,-1],[1,-1,-1]])
    xnodes_ndt = np.zeros((N+1,8,3))
    xnodes_ndt[0,:,:] = nodes_t0
    unodes_ndt = np.zeros((N+1,8,3))
    
    point_ndt = np.zeros((N+1,3))
    point_ndt[0,:] = P
    grad_u_P = np.zeros((N+1,3,3))
    F_P = np.zeros((N+1,3,3))
    C_P = np.zeros((N+1,3,3))
    # b_P = np.zeros((N+1,3,3))
    l_P = np.zeros((N+1,3,3))
    d_P = np.zeros((N+1,3,3))
    E_P = np.zeros((N+1,3,3))
    e_P = np.zeros((N+1,3,3))
    tau_P = np.zeros((N+1,3,3))
    S_P = np.zeros((N+1,3,3))
    sigma_P = np.zeros((N+1,3,3))
    Eeq_P = np.zeros((N+1))
    eeq_P = np.zeros((N+1))
    Seq_P = np.zeros((N+1))
    sigmaeq_P = np.zeros((N+1))


    for n in range(N):
        t = n*dt
        # Calculation of nodal positions and displacements
        for i in range(8):
            X1 = xnodes_ndt[0,i,0]
            X2 = xnodes_ndt[0,i,1]
            X3 = xnodes_ndt[0,i,2]
            xnodes_ndt[n+1,i,0] = eval(x1)
            xnodes_ndt[n+1,i,1] = eval(x2)
            xnodes_ndt[n+1,i,2] = eval(x3)
            unodes_ndt[n+1,i,0] = xnodes_ndt[n+1,i,0] - X1
            unodes_ndt[n+1,i,1] = xnodes_ndt[n+1,i,1] - X2
            unodes_ndt[n+1,i,2] = xnodes_ndt[n+1,i,2] - X3
            
        # Calculation of required quantities at point 'P'
        X1 = P[0]
        X2 = P[1]
        X3 = P[2]
        point_ndt[n+1,0] = eval(x1)
        point_ndt[n+1,1] = eval(x2)
        point_ndt[n+1,2] = eval(x3)
        NX1 = np.array([N1X1(X2,X3), N2X1(X2,X3), N3X1(X2,X3), N4X1(X2,X3), N5X1(X2,X3), N6X1(X2,X3), N7X1(X2,X3), N8X1(X2,X3)])
        NX2 = np.array([N1X2(X1,X3), N2X2(X1,X3), N3X2(X1,X3), N4X2(X1,X3), N5X2(X1,X3), N6X2(X1,X3), N7X2(X1,X3), N8X2(X1,X3)])
        NX3 = np.array([N1X3(X1,X2), N2X3(X1,X2), N3X3(X1,X2), N4X3(X1,X2), N5X3(X1,X2), N6X3(X1,X2), N7X3(X1,X2), N8X3(X1,X2)])
        NX  = np.array([NX1, NX2, NX3])
        for i in range(3):
            for j in range(3):
                grad_u_P[n+1,i,j] = np.dot(unodes_ndt[n+1,:,i],NX[j])
        F_P[n+1,:,:] = I + grad_u_P[n+1,:,:]
        dF_P = F_P[n+1,:,:]-F_P[n,:,:]
        # dE_P = 0.5*(np.dot(dF_P.T,F_P[n,:,:])+np.dot(F_P[n,:,:]).T,dF_P)
        dE_P = 0.5*(np.dot(dF_P.T,F_P[n+1,:,:])+np.dot(F_P[n+1,:,:].T,dF_P)) # alternate
        C_P[n+1,:,:] = np.dot(F_P[n+1,:,:].T,F_P[n+1,:,:])
        J = math.sqrt(np.linalg.det(C_P[n+1,:,:]))
        invC_P = np.linalg.inv(C_P[n+1,:,:])
        # t4one = np.zeros((3,3,3,3))
        # for i in range(3):
        #     for j in range(3):
        #         for k in range(3):
        #             for l in range(3):
        #                 t4one[i][j][k][l] = invC_P[i][j]*invC_P[k][l]
        # t4two = np.zeros((3,3,3,3))
        # for i in range(3):
        #     for j in range(3):
        #         for m in range(3):
        #             for n in range(3):
        #                 t4two[i][j][m][n] = 0.5*(invC_P[i][m]*invC_P[n][j]+invC_P[i][n]*invC_P[m][j])
        t4one = np.einsum('ij,kl->ijkl', invC_P, invC_P)
        t4two = 0.5*(np.einsum('im,nj->ijmn', invC_P, invC_P) + np.einsum('in,mj->ijmn', invC_P, invC_P))
        C4 = lmda*t4one + 2*(mu-lmda*math.log(J))*t4two
        dS_P = np.einsum('ijkl,kl->ij', C4, dE_P)
        E_P[n+1,:,:] = E_P[n,:,:] + dE_P
        S_P[n+1,:,:] = S_P[n,:,:] + dS_P

        dl_P = np.dot(dF_P,np.linalg.inv(F_P[n+1,:,:]))
        # print(dl_P)
        l_P[n+1,:,:] = l_P[n,:,:] + dl_P
        dd_P = (dl_P + dl_P.T)/2
        d_P[n+1,:,:] = (l_P[n+1,:,:] + l_P[n+1,:,:].T)/2
        # d_P[n+1,:,:] = d_P[n,:,:] + dd_P
        de_P = dd_P - np.dot(dl_P.T,e_P[n,:,:]) + np.dot(e_P[n,:,:],dl_P.T)
        e_P[n+1,:,:] = e_P[n,:,:] + de_P
        c4 = (1/J)*np.einsum('im,jn,ko,lp,mnop->ijkl',F_P[n+1,:,:],F_P[n+1,:,:],F_P[n+1,:,:],F_P[n+1,:,:],C4)
        dtau_P = J*dt*np.einsum('ijkl,kl->ij', c4, d_P[n+1,:,:]) + np.dot(dl_P,tau_P[n,:,:]) + np.dot(tau_P[n,:,:],dl_P.T)
        # print(J*dt*np.einsum('ijkl,kl->ij', c4, d_P[n+1,:,:]))
        dsigma_P = (1/J)*dtau_P
        tau_P[n+1,:,:] = tau_P[n,:,:] + dtau_P
        sigma_P[n+1,:,:] = sigma_P[n,:,:] + dsigma_P
        

        Eeq_P[n+1] = eqstrain(E_P[n+1,:,:],nu)
        eeq_P[n+1] = eqstrain(e_P[n+1,:,:],nu)
        Seq_P[n+1] = eqstress(S_P[n+1,:,:])
        sigmaeq_P[n+1] = eqstress(sigma_P[n+1,:,:])

    # DataFrame
    dN = round(N/round(T))
    output = pd.DataFrame({'Time[sec]':range(round(T)+1), 'Eeq': Eeq_P[0:N+1:dN], 'Seq': Seq_P[0:N+1:dN], 'eeq': eeq_P[0:N+1:dN], 'sigmaeq': sigmaeq_P[0:N+1:dN]}).set_index('Time[sec]')
    display(output)
    
    # # create a figure and axes
    # fig = plt.figure(figsize=(16,8), dpi=100)
    # dmap = plt.subplot(1,2,1)   
    # SvE = plt.subplot(1,2,2)
    # sve = SvE.twinx().twiny()

    # # set up the subplots as needed
    # dmap.set_xlim(limit[0], limit[1])            
    # dmap.set_ylim(limit[2], limit[3])
    # dmap.axvline(c="black",lw=1,ls='--')
    # dmap.axhline(c="black",lw=1,ls='--')
    # dmap.set_aspect('equal')
    # dmap.set_xlabel('x1')
    # dmap.set_ylabel('x2')
    # # dmap.set_title('Deformation Map')

    # # Second-Piola Kirchhoff's stress vs Lagrange strain
    # lxlim = round(Eeq_P.min(),2)
    # rxlim = round(Eeq_P.max(),2)
    # lylim = round(Seq_P.min(),2)
    # rylim = round(Seq_P.max(),2)
    # if lxlim!=0 or rxlim!=0:
    #     if lxlim == 0:
    #         lxlim = -(rxlim/10)
    #     SvE.set_xlim(lxlim,rxlim)    
    # if lylim!=0 or rylim!=0:
    #     if lylim == 0:
    #         lylim = -(rylim/10)
    #     SvE.set_ylim(lylim,rylim)
    # SvE.axvline(c="black",lw=1,ls='--')
    # SvE.axhline(c="black",lw=1,ls='--')
    # SvE.set_box_aspect(1)
    # # SvE.set_adjustable('datalim')
    # # SvE.axis('tight')
    # # SvE.set_xlabel(r'$||e||$ or $||E||$')
    # # SvE.set_ylabel(r'$||\sigma||$ or $||S||$')
    # SvE.set_xlabel('Strain')
    # SvE.set_ylabel('Stress')
    # # SvE.set_xlabel(r'$\|\underline{\underline{e}}\|$ / $\|underline{\underline{E}}\|$', fontsize=16)
    # # SvE.set_ylabel(r'$\|\underline{\underline{\sigma}}\|$ / $\|\underline{\underline{S}}\|$', fontsize=16)
    # SvE.set_title("Stress vs Strain at 'P'", y=1, pad=35)

    # # Cauchy stress vs Euler strain
    # lxlim = round(eeq_P.min(),2)
    # rxlim = round(eeq_P.max(),2)
    # lylim = round(sigmaeq_P.min(),2)
    # rylim = round(sigmaeq_P.max(),2)
    # if lxlim!=0 or rxlim!=0:
    #     if lxlim == 0:
    #         lxlim = -(rxlim/10)
    #     sve.set_xlim(lxlim,rxlim)    
    # if lylim!=0 or rylim!=0:
    #     if lylim == 0:
    #         lylim = -(rylim/10)
    #     sve.set_ylim(lylim,rylim)
    # # sve.set_xlabel(r'$||e||$')
    # # sve.set_ylabel(r'$||\sigma||$')
    

    # # create objects that will change in the animation.
    # # These are initially empty, and will be given new values for each frame in the animation.
    # timestep = dmap.set_title('', y=1, pad=35)
    # face5687, = dmap.plot([], [], c="grey", lw=1, ls='--')
    # edge15, = dmap.plot([], [], c="grey", lw=1, ls='--')
    # edge26, = dmap.plot([], [], c="grey", lw=1, ls='--')
    # edge48, = dmap.plot([], [], c="grey", lw=1, ls='--')
    # edge37, = dmap.plot([], [], c="grey", lw=1, ls='--')
    # face1243, = dmap.plot([], [], 'tab:blue', lw=2.5)
    # point, = dmap.plot([], [],'ob', ms=8, label="The point 'P'")
    # dmap.legend(loc='upper right')

    # SeqvEeq, = SvE.plot([], [], 'tab:blue', lw=2.5, label=r'$||S||$ vs $||E||$')
    # SvEpoint, = SvE.plot([], [], 'tab:blue', marker='o', ms=8)
    # SvElinex, = SvE.plot([], [], 'tab:blue', lw=1.5, ls='--')
    # SvEliney, = SvE.plot([], [], 'tab:blue', lw=1.5, ls='--')
    # SvE.legend(loc='upper left')
    # sigmaVe, = sve.plot([], [], 'tab:orange', lw=2.5, label=r'$||\sigma||$ vs $||e||$')
    # svepoint, = sve.plot([], [], 'tab:orange', marker='o', ms=8)
    # svelinex, = sve.plot([], [], 'tab:orange', lw=1.5, ls='--')
    # sveliney, = sve.plot([], [], 'tab:orange', lw=1.5, ls='--')
    # sve.legend(loc='upper right')

    # # animation function. This is called sequentially
    # def drawframe(n, dt=dt, x1x2=xnodes_ndt, S=Seq_P, E=Eeq_P, sigma=sigmaeq_P, e=eeq_P, P=point_ndt):
    #     face5687.set_data(x1x2[n,(4,5,7,6,4),0], x1x2[n,(4,5,7,6,4),1])
    #     edge15.set_data(x1x2[n,(0,4),0], x1x2[n,(0,4),1])
    #     edge26.set_data(x1x2[n,(1,5),0], x1x2[n,(1,5),1])
    #     edge48.set_data(x1x2[n,(3,7),0], x1x2[n,(3,7),1])
    #     edge37.set_data(x1x2[n,(2,6),0], x1x2[n,(2,6),1])
    #     face1243.set_data(x1x2[n,(0,1,3,2,0),0], x1x2[n,(0,1,3,2,0),1])
    #     point.set_data(point_ndt[n,0],point_ndt[n,1])
    #     timestep.set_text('Deformed config. at t = {:04.1f} sec'.format(round(n*dt,1)))

    #     SeqvEeq.set_data(E[:n+1],S[:n+1])
    #     SvEpoint.set_data(E[n],S[n])
    #     SvElinex.set_data([E[n],-1e100],[S[n],S[n]])
    #     SvEliney.set_data([E[n],E[n]],[S[n],-1e100])
    #     sigmaVe.set_data(e[:n+1],sigma[:n+1])
    #     svepoint.set_data(e[n],sigma[n])
    #     svelinex.set_data([e[n],1e100],[sigma[n],sigma[n]])
    #     sveliney.set_data([e[n],e[n]],[sigma[n],1e100])
    #     return (face1243,face5687)
    
    # anim = animation.FuncAnimation(fig, drawframe, frames=N+1, interval=20/animx, blit=True)
    # return(anim)

## 1. Translation
$$
\begin{bmatrix}
    x_1 \\ x_2 \\ x_3
\end{bmatrix}
=
\begin{bmatrix}
    X_1 \\ X_2 \\ X_3
\end{bmatrix}
+
\begin{bmatrix}
    t \\ t \\ 0
\end{bmatrix}
\implies
\begin{align*}
    x_1 &= X_1+t\\
    x_2 &= X_2+t\\
    x_3 &= X_3
\end{align*}
$$

In [136]:
x1 = 'X1+t'
x2 = 'X2+t'
x3 = 'X3'
postproc(x1, x2, x3, T=10, P = [1,1,1], limit=[-2, 12, -2, 12], animx=1)


,Eeq,Seq,eeq,sigmaeq
Time[sec],,,,
0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,0.000000e+00,2.851581e-10,0.000000e+00,2.851581e-10
2,0.000000e+00,2.851581e-10,0.000000e+00,5.703163e-10
3,0.000000e+00,2.851581e-10,0.000000e+00,1.140633e-09
4,0.000000e+00,2.851581e-10,0.000000e+00,0.000000e+00
5,0.000000e+00,2.851581e-10,0.000000e+00,0.000000e+00
6,0.000000e+00,2.851581e-10,0.000000e+00,2.281265e-09
7,0.000000e+00,2.851581e-10,0.000000e+00,2.281265e-09
8,3.700743e-16,4.032745e-10,3.700743e-16,1.310487e-08


## 2. Rotation
$$
\begin{bmatrix}
    x_1 \\ x_2 \\ x_3
\end{bmatrix}
=
\begin{bmatrix}
    \cos\omega t & -\sin\omega t & 0 \\
    \sin\omega t & \cos\omega t & 0 \\
    0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
    X_1 \\ X_2 \\ X_3
\end{bmatrix}
\implies
\begin{align*}
    x_1 &= X_1\cos\omega t - X_2\sin\omega t \\
    x_2 &= X_1\sin\omega t + X_2\cos\omega t \\
    x_3 &= X_3
\end{align*}
$$

$$
\omega t = 90^o \implies \omega = 9^o
$$

In [137]:
x1 = 'X1*math.cos(math.radians(9*t))-X2*math.sin(math.radians(9*t))'
x2 = 'X1*math.sin(math.radians(9*t))+X2*math.cos(math.radians(9*t))'
x3 = 'X3'
postproc(x1, x2, x3, T=10, P = [1,1,1], limit=[-7, 7, -7, 7], animx=1.5)

,Eeq,Seq,eeq,sigmaeq
Time[sec],,,,
0,0.000000,0.000000,0.000000,0.000000
1,0.000488,0.000488,0.000488,977.157408
2,0.001002,0.001002,0.001002,4013.688620
3,0.001516,0.001516,0.001516,9112.981814
4,0.002030,0.002030,0.002030,16278.429352
5,0.002544,0.002544,0.002544,25513.427783
6,0.003059,0.003059,0.003059,36821.377849
7,0.003573,0.003573,0.003573,50205.684488
8,0.004087,0.004087,0.004087,65669.756845


In [138]:
x1 = 'X1*math.cos(math.radians(9*t))-X2*math.sin(math.radians(9*t))'
x2 = 'X1*math.sin(math.radians(9*t))+X2*math.cos(math.radians(9*t))'
x3 = 'X3'
postproc(x1, x2, x3, T=10, P = [1,1,1], limit=[-7, 7, -7, 7], animx=1.5)

,Eeq,Seq,eeq,sigmaeq
Time[sec],,,,
0,0.000000,0.000000,0.000000,0.000000
1,0.000488,0.000488,0.000488,977.157408
2,0.001002,0.001002,0.001002,4013.688620
3,0.001516,0.001516,0.001516,9112.981814
4,0.002030,0.002030,0.002030,16278.429352
5,0.002544,0.002544,0.002544,25513.427783
6,0.003059,0.003059,0.003059,36821.377849
7,0.003573,0.003573,0.003573,50205.684488
8,0.004087,0.004087,0.004087,65669.756845


## 3. Translation + Rotation
$$
\begin{bmatrix}
    x_1 \\ x_2 \\ x_3
\end{bmatrix}
=
\begin{bmatrix}
    \cos\omega t & -\sin\omega t & 0 \\
    \sin\omega t & \cos\omega t & 0 \\
    0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
    X_1 \\ X_2 \\ X_3
\end{bmatrix}
+
\begin{bmatrix}
    t \\ t \\ 0
\end{bmatrix}
\implies
\begin{align*}
    x_1 &= X_1\cos\omega t - X_2\sin\omega t + t\\
    x_2 &= X_1\sin\omega t + X_2\cos\omega t + t\\
    x_3 &= X_3
\end{align*}
$$
$$
\omega t = 90^o \implies \omega = 9^o
$$

In [139]:
x1 = 'X1*math.cos(math.radians(9*t))-X2*math.sin(math.radians(9*t))+t'
x2 = 'X1*math.sin(math.radians(9*t))+X2*math.cos(math.radians(9*t))+t'
x3 = 'X3'
postproc(x1, x2, x3, T=10, P = [1,1,1], limit=[-2, 12, -2, 12], animx=1)

,Eeq,Seq,eeq,sigmaeq
Time[sec],,,,
0,0.000000,0.000000,0.000000,0.000000
1,0.000488,0.000488,0.000488,977.157408
2,0.001002,0.001002,0.001002,4013.688620
3,0.001516,0.001516,0.001516,9112.981814
4,0.002030,0.002030,0.002030,16278.429352
5,0.002544,0.002544,0.002544,25513.427783
6,0.003059,0.003059,0.003059,36821.377849
7,0.003573,0.003573,0.003573,50205.684488
8,0.004087,0.004087,0.004087,65669.756845


## 4. Pure Shear
$$
\begin{align*}
    x_1 &= X_1 + X_2t\\
    x_2 &= X_2\\
    x_3 &= X_3
\end{align*}
$$

In [140]:
x1 = 'X1+X2*t'
x2 = 'X2'
x3 = 'X3'
postproc(x1, x2, x3, T=10, P = [1,1,1], limit=[-7, 7, -7, 7], animx=1)

,Eeq,Seq,eeq,sigmaeq
Time[sec],,,,
0,0.000000,0.000000,0.000000,0.000000e+00
1,0.636764,0.791667,0.501222,1.451150e+06
2,3.532740,2.149673,1.862213,6.973845e+06
3,14.225331,4.257958,9.721653,1.914347e+07
4,41.479572,7.174021,32.830901,4.113401e+07
5,97.563724,10.913724,83.207557,7.625989e+07
6,198.291570,15.482391,176.474393,1.278597e+08
7,363.016853,20.882151,331.804891,1.992739e+08
8,614.630604,27.113981,571.912202,2.938410e+08


## 5. Generic
$$
\begin{align*}
    x_1 &= X_1e^t + X_3(e^t-1)\\
    x_2 &= X_2 + X_3(e^t-e^{-t})\\
    x_3 &= X_3
\end{align*}
$$

In [141]:
x1 = 'X1*math.exp(t)+X3*(math.exp(t)-1)'
x2 = 'X2+X3*(math.exp(t)-math.exp(-t))'
x3 = 'X3'
postproc(x1, x2, x3, T=10, P = [1,1,1], limit=[-7, 7, -7, 7], animx=0.5)

,Eeq,Seq,eeq,sigmaeq
Time[sec],,,,
0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,3.696740e+00,3.831513e+06,2.443620e+00,2.191306e+06
2,3.380620e+02,5.640132e+07,6.767294e+01,1.112976e+07
3,2.030269e+04,6.199977e+08,4.330167e+03,3.690415e+07
4,1.123572e+06,6.107239e+09,2.509680e+05,9.104336e+07
5,6.145961e+07,5.646077e+10,1.387044e+07,1.595921e+08
6,3.356436e+09,5.010311e+11,7.589554e+08,6.472744e+08
7,1.832615e+11,4.321765e+12,4.145278e+10,4.418649e+09
8,1.000579e+13,3.651232e+13,2.263379e+12,2.261197e+10


## 6. Biaxial Tension
$$
\begin{align*}
    x_1 &= X_1 + X_1t\\
    x_2 &= X_2 + X_2t\\
    x_3 &= X_3
\end{align*}
$$

In [142]:
x1 = 'X1+X1*t'
x2 = 'X2+X2*t'
x3 = 'X3'
postproc(x1, x2, x3, T=10, P = [1,1,1], limit=[-7, 7, -7, 7], animx=1)

,Eeq,Seq,eeq,sigmaeq
Time[sec],,,,
0,0.000000,0.000000e+00,0.000000,0.000000e+00
1,1.187500,5.614220e+05,0.546503,3.353130e+05
2,3.250000,1.073603e+06,0.887887,4.148585e+05
3,6.145833,1.435400e+06,1.129366,2.650773e+05
4,9.875000,1.709749e+06,1.316365,8.059878e+03
5,14.437500,1.929648e+06,1.468996,2.972691e+05
6,19.833333,2.112865e+06,1.597952,6.223604e+05
7,26.062500,2.269817e+06,1.709601,9.527522e+05
8,33.125000,2.407077e+06,1.808044,1.280891e+06


## 7. Biaxial Compression
$$
\begin{align*}
    x_1 &= X_1 - X_1t\\
    x_2 &= X_2 - X_2t\\
    x_3 &= X_3
\end{align*}
$$

In [143]:
x1 = 'X1-X1*t'
x2 = 'X2-X2*t'
x3 = 'X3'
postproc(x1, x2, x3, T=0.8, P = [1,1,1], limit=[-7, 7, -7, 7], animx=1.2)

,Eeq,Seq,eeq,sigmaeq
Time[sec],,,,
0,0.000,0.000000e+00,0.000000,0.000000e+00
1,0.398,4.177108e+07,1.331225,1.534990e+07


## 8. Plane Strain Compression
$$
\begin{align*}
    x_1 &= X_1 + X_1t\\
    x_2 &= X_2 - X_2t\\
    x_3 &= X_3
\end{align*}
$$

In [144]:
x1 = 'X1+X1*t'
x2 = 'X2-X2*t'
x3 = 'X3'
postproc(x1, x2, x3, T=0.8, P = [1,1,1], limit=[-7, 7, -7, 7], animx=1.2)

,Eeq,Seq,eeq,sigmaeq
Time[sec],,,,
0,0.000000,0.000000e+00,0.00000,0.000000e+00
1,1.378478,1.342318e+07,1.89614,3.523759e+06
